In [1]:
%pip install -q scikit-learn seaborn matplotlib pandas numpy
import sys, sklearn, seaborn, matplotlib, pandas as pd, numpy as np
print('Python:', sys.executable)
print('sklearn:', sklearn.__version__)
print('seaborn:', seaborn.__version__)
print('matplotlib:', matplotlib.__version__)


Note: you may need to restart the kernel to use updated packages.
Python: c:\ProgramData\anaconda3\python.exe
sklearn: 1.6.1
seaborn: 0.13.2
matplotlib: 3.10.0


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df=pd.read_csv(r'C:\AI workforce\ML-flow\modified_data.csv')
df

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,0,2,3,8,0.974587,26.16,0.093267,7080,1.259701,-0.12356,-0.12356,-0.374166,59.0,0.940725
1,1/1/2015,0,1,3,1,-0.075500,3.94,0.237837,960,-0.978354,-0.12356,-0.12356,-0.374166,8.0,0.886500
2,1/1/2015,0,2,3,11,0.974587,11.41,-0.173048,3660,0.989084,-0.12356,-0.12356,-0.374166,30.5,0.800570
3,1/1/2015,0,2,3,12,0.974587,11.41,-0.173048,3660,0.989084,-0.12356,-0.12356,-0.374166,30.5,0.800570
4,1/1/2015,0,2,3,6,0.974587,25.90,0.202719,1920,0.989084,-0.12356,-0.12356,-0.374166,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,1,0,5,10,-0.075500,2.90,0.237837,960,-0.978354,-0.12356,-0.12356,-0.374166,8.0,0.628333
1193,3/11/2015,1,0,5,8,-0.781026,3.90,0.237837,960,-0.978354,-0.12356,-0.12356,-0.374166,8.0,0.625625
1194,3/11/2015,1,0,5,7,-1.249547,3.90,0.237837,960,-0.978354,-0.12356,-0.12356,-0.374166,8.0,0.625625
1195,3/11/2015,1,0,5,9,-0.075500,2.90,0.237837,1800,-0.978354,-0.12356,-0.12356,-0.374166,15.0,0.505889


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X = df.drop(columns=['actual_productivity'])
y = df['actual_productivity']
X = X.select_dtypes(include=[np.number])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
model = RandomForestRegressor(n_estimators=200,max_depth=30,random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Random Forest Regression Model:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

Random Forest Regression Model:
R² Score: 0.5680
RMSE: 0.1071


In [5]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(
    n_estimators=300,    
    learning_rate=0.05,  
    max_depth=5,         
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42
)

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("Gradient Boosting Model Performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")

Gradient Boosting Model Performance:
R² Score: 0.5320
RMSE: 0.1115


In [6]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import uniform, randint

param_dist = {
    'n_estimators': randint(500, 1500),      
    'learning_rate': uniform(0.01, 0.2),      
    'max_depth': randint(3, 10),              
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'subsample': uniform(0.7, 0.3),           
    'max_features': ['sqrt', 'log2', None]
}
gb = GradientBoostingRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=gb,
    param_distributions=param_dist,
    n_iter=50,             
    cv=3,                  
    n_jobs=-1,
    verbose=2,
    scoring='r2',
    random_state=42
)

random_search.fit(X_train, y_train)

best_gb = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

y_pred = best_gb.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\nTuned Gradient Boosting Performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters: {'learning_rate': np.float64(0.01661014658010968), 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 552, 'subsample': np.float64(0.8468358280832688)}

Tuned Gradient Boosting Performance:
R² Score: 0.4637
RMSE: 0.1193


In [7]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("XGBoost Model Performance:")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")


XGBoost Model Performance:
R² Score: 0.5318
RMSE: 0.1115
